# Chromalink Midi
A Python script to control ChromaLink lights such as Yeelight 1S to make a lightshow in sync with the beats of the Mixxx DJ software.

By Stephen Karl Larroque

License: MIT (opensource)

v0.1

Tested on Windows 10 with Python 3

In [1]:
from ChromaPython import ChromaApp, ChromaAppInfo, ChromaColor, Colors, ChromaGrid # from https://github.com/chroma-sdk/chroma-python
# TODO: see https://github.com/ignis-sec/better-chroma-python and https://github.com/d-rez/chroma-python/commit/f905e7edcb9fa5dc5f423d849f80350c05ce0616 -- but those don't support chromalink... but could merge the changes there...
# You need to install and launch Yeelight Chroma Connector and Razer Synapse first, and enable this app in Razer Connector after running this cell at least once.
from time import sleep

Info = ChromaAppInfo()
Info.DeveloperName = 'Stephen Larroque'
Info.DeveloperContact = 'na'
Info.Category = 'application'
Info.SupportedDevices = ['chromalink']
Info.Description = 'ChromaLink devices controlled by virtual midi.'
Info.Title = 'ChromaLink Midi'

App = ChromaApp(Info)

sleep(2)

In [2]:
# Do not forget to launch both the Yeelight Chroma Connector AND Razer Synapse (and activate this app in the Connector tab)
colors_list = [Colors.RED, Colors.BLUE, Colors.GREEN, Colors.MAGENTA] # most musics are 4 beats so try to keep a multiple of 4 in the number of colors
print('TESTING CHROMALINK DEVICES')
for i in range(10):
    print(App.ChromaLink.setStatic(colors_list[i % len(colors_list)]))
    sleep(0.2)

TESTING CHROMALINK DEVICES
(True, 0)
(True, 0)
(True, 0)
(True, 0)
(True, 0)
(True, 0)
(True, 0)
(True, 0)
(True, 0)
(True, 0)


In [3]:
# Alternative to Chroma: directly control yeelight devices using python-yeelight: https://yeelight.readthedocs.io/en/latest/

In [4]:
# Please install python-rtmidi and mido using pip install first! Do not try to pip install rtmidi, it is unnecessary!
# Then modify the midi loopback channel name below to what you set in loopmidi or loopbe1

# Open midi port (the with statement does not work well in Jupyter notebook, the kernel freezes on close...)
# But we need to open the port only once, otherwise we will get an error
import mido
import time

# First create a virtual port on Windows 10 using loopMidi or LoopBe1. On Linux and MacOSX, the virtual port can be created here directly. https://spotlightkid.github.io/python-rtmidi/rtmidi.html#rtmidi.MidiIn.open_virtual_port
# Note: prefer to use a different software for each port, eg, if you use Harmonic Mixing controller mapping for AutoDJ, use it via loopMidi, and set the Midi for light controller on LoopBe1. Otherwise, for some reasons, loopMidi will mix up signals between different ports (or maybe it's Mixxx...)
# Secondly, activate the port in Mixxx, by loading the MIDI for light controller mapping for the virtual port.
# Then, launch this script.
# Documentation: https://github.com/mixxxdj/mixxx/wiki/midi_clock_output

print(mido.get_input_names())

port = "loopMIDI Port Light 3"

inport = mido.open_input(port)
print("Port opened with success!")

['LoopBe Internal MIDI 0', 'TouchOSC Bridge 1', 'loopMIDI Port OSC 2', 'loopMIDI Port Light 3', 'loopMIDI Port AutoDJ 4']
Port opened with success!


In [7]:
# Parameters
lightshow_duration = 600 # in seconds
debug = False

In [8]:
print("Restart notebook to stop the loop or if no message gets printed.\n")
print("Note: don't worry if you see the lights flashing too quickly at first, it will start getting in rhythm after a few seconds.")

#with mido.open_input(port, virtual=True) as inport:
end_of_loop = time.time() + lightshow_duration  # work for x seconds then stop (otherwise we get an infinite loop that can't be interrupted in Jupyter)

try:
    while time.time() < end_of_loop :
        msg = inport.receive()
        #print(msg)
        if hasattr(msg, 'note') and msg.note == 50 and msg.type == 'note_on' and msg.velocity == 100:
            i += 1
            i = i % len(colors_list) # to avoid overflow...
            if debug:
                print(App.ChromaLink.setStatic(colors_list[i % len(colors_list)]))
                print("Beat!")
                print(msg)
            else:
                App.ChromaLink.setStatic(colors_list[i % len(colors_list)])
except KeyboardInterrupt:
    pass

Restart notebook to stop the loop or if no message gets printed.

Note: don't worry if you see the lights flashing too quickly at first, it will start getting in rhythm after a few seconds.


---------------
DEBUG

In [ ]:
port = "LoopBe Internal MIDI" # Prompts user for MIDI input port, unless a valid port number or name
                                                  # is given as the first argument on the command line.
                                                  # API backend defaults to ALSA on Linux.

In [ ]:
# https://github.com/RaddedMC/ChromaMidi
import math
import sys
from rtmidi.midiutil import open_midiinput

#port = sys.argv[1] if len(sys.argv) > 1 else None # Prompts user for MIDI input port, unless a valid port number or name
                                                  # is given as the first argument on the command line.
                                                  # API backend defaults to ALSA on Linux.
midiin, port_name = open_midiinput(port)
print("Midi opened on device " + str(port_name) + ".")
print("---------------------------------------")

while True:
    msg = midiin.get_message()
    
    if msg:
        led_num = msg[0][1]-48
        colors = (0,0,0)
        if msg[0][0] == 144:
            # ON
            velocity = msg[0][2]
            print("%r: Light %i set to velocity %s" % (msg, led_num, velocity))
           # setALight(abs(led_num), colors[0], colors[1], colors[2])
        elif msg[0][0] == 128:
            # OFF
            print("%r: Turning off light %i" % (msg, led_num))
            #setALight(abs(led_num), colors[0], colors[1], colors[2])

In [ ]:
class CleanExit(object):
    def __enter__(self):
        return self
    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is KeyboardInterrupt:
            return True
        else:
            return exc_type is None

def signal_handler(signal, frame):
    print('You pressed Ctrl+C!')
    raise KeyboardInterrupt

In [ ]:
import signal

import mido

print(mido.get_input_names())

port = "loopMIDI-lights 1"

print("Restart notebook to stop the loop.")

#with mido.open_input(port, virtual=True) as inport:
with mido.open_input(port) as inport:
    try:
        while True:
            msg = inport.receive()
            print(msg)
    except KeyboardInterrupt:
        pass

In [ ]:
inport = mido.open_input()

In [ ]:
mido.get_input_names()